In [1]:
import os
from langchain.document_loaders import TextLoader, CSVLoader, PyPDFLoader, UnstructuredWordDocumentLoader
from datetime import datetime
from langchain.schema import Document
from langchain.text_splitter import MarkdownHeaderTextSplitter, CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()



True

In [2]:
import os
from langchain_community.document_loaders import CSVLoader, PyPDFLoader, UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from datetime import datetime
from langchain.schema import Document

# ------------------ Loaders with Splitters ------------------

def load_csv(file_path):
    loader = CSVLoader(file_path)
    return loader.load()

def load_markdown(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=150,
        chunk_overlap=15,
        separators=["\n\n", "\n", " "]
    )
    chunks = splitter.split_text(text)
    return [Document(page_content=chunk, metadata={"source": file_path}) for chunk in chunks]

def load_text(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=30,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    chunks = splitter.split_text(text)
    return [Document(page_content=chunk, metadata={"source": file_path}) for chunk in chunks]

def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=30,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.split_documents(pages)

def load_docx(file_path):
    loader = UnstructuredWordDocumentLoader(file_path)
    pages = loader.load()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.split_documents(pages)

# Loading documents by role

def load_documents_by_role(role: str):
    base_path = "./resources/data"
    role_paths = {
        "finance": ["finance"],
        "marketing": ["marketing"],
        "engineering": ["engineering"],
        "human_resource": ["hr"],
        "employee": ["general"],
    }

    folders = role_paths.get(role, ["general"])
    print("folders: ", folders)
    docs = []

    for folder in folders:
        path = os.path.join(base_path, folder)
        print("path: ", path)
        if os.path.exists(path):
            for file_name in os.listdir(path):
                file_path = os.path.join(path, file_name)
                print("file_path:", file_path)

                try:
                    loaded_docs = []
                    if file_name.endswith(".csv"):
                        loaded_docs = load_csv(file_path)
                        print(file_name, "CSV Documents loaded :" , file_path)
                        print("Number of Documents loaded: ",len(loaded_docs))
                        print("------------------------------------------------------------------------------------------------------------")

                    elif file_name.endswith(".md"):
                        loaded_docs = load_markdown(file_path)
                        print(file_name, "MD Documents loaded", file_path)
                        print("Number of Documents loaded: ",len(loaded_docs))
                        print("------------------------------------------------------------------------------------------------------------")

                    elif file_name.endswith(".txt"):
                        loaded_docs = load_text(file_path)
                        print(file_name,"Text Documents loaded", file_path)

                    elif file_name.endswith(".pdf"):
                        loaded_docs = load_pdf(file_path)
                        print(file_name,"PDF Documents loaded", file_path)

                    elif file_name.endswith(".docx"):
                        loaded_docs = load_docx(file_path)
                        print(file_name,"Word Documents loaded", file_path)

                    else:
                        continue

                    # Add metadata
                    for doc in loaded_docs:
                        doc.metadata["source"] = file_path
                        doc.metadata["role"] = folder
                        doc.metadata["last_modified"] = datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y-%m-%d %H:%M:%S')

                    docs.extend(loaded_docs)

                except Exception as e:
                    print(f"Failed to load {file_path}: {e}")
    print("Total docs loaded : ",len(docs))
    return docs

roles = ["finance", "human_resource", "marketing", "engineering", "employee"]
def get_documents_by_role():
    documents_by_role = {}
    for role in roles:
        print("===============================================================================================================")
        print(f"Loading document for {role} ...... ")
        documents_by_role[role] = load_documents_by_role(role)
        print(f"Loading document for {role} complete")
    return documents_by_role    
    
role_documents = get_documents_by_role()


Loading document for finance ...... 
folders:  ['finance']
path:  ./resources/data\finance
file_path: ./resources/data\finance\financial_summary.md
financial_summary.md MD Documents loaded ./resources/data\finance\financial_summary.md
Number of Documents loaded:  73
------------------------------------------------------------------------------------------------------------
file_path: ./resources/data\finance\quarterly_financial_report.md
quarterly_financial_report.md MD Documents loaded ./resources/data\finance\quarterly_financial_report.md
Number of Documents loaded:  116
------------------------------------------------------------------------------------------------------------
Total docs loaded :  189
Loading document for finance complete
Loading document for human_resource ...... 
folders:  ['hr']
path:  ./resources/data\hr
file_path: ./resources/data\hr\hr_data.csv
hr_data.csv CSV Documents loaded : ./resources/data\hr\hr_data.csv
Number of Documents loaded:  100
-----------------

In [3]:
embedding=OpenAIEmbeddings()
persist_directory="./.chroma"


def create_vector_store(docs, collection_name):
    print(f"Creating Chroma vectorstore for collection: {collection_name} ...... ")

    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embedding,
        collection_name=collection_name,
        persist_directory=persist_directory
    )
    print(f"Creating Chroma vectorstore for collection: {collection_name} success")
    print("-----------------------------------------------------------------------------------------------------------------")
    # return vectorstore

for role , docs in role_documents.items():
    print(role.upper(), ":-")
    print("Total Documents passed: ",len(docs))
    create_vector_store(docs=docs, collection_name=role)

FINANCE :-
Total Documents passed:  189
Creating Chroma vectorstore for collection: finance ...... 
Creating Chroma vectorstore for collection: finance success
-----------------------------------------------------------------------------------------------------------------
HUMAN_RESOURCE :-
Total Documents passed:  100
Creating Chroma vectorstore for collection: human_resource ...... 
Creating Chroma vectorstore for collection: human_resource success
-----------------------------------------------------------------------------------------------------------------
MARKETING :-
Total Documents passed:  330
Creating Chroma vectorstore for collection: marketing ...... 
Creating Chroma vectorstore for collection: marketing success
-----------------------------------------------------------------------------------------------------------------
ENGINEERING :-
Total Documents passed:  283
Creating Chroma vectorstore for collection: engineering ...... 
Creating Chroma vectorstore for collection:

In [7]:
retrievers = {}

In [16]:
def create_retriever_by_role(role):

   return Chroma(persist_directory=persist_directory, 
                        embedding_function=embedding, 
                        collection_name = role).as_retriever()

In [17]:
for role in roles:
    retrievers[role] = create_retriever_by_role(role)

In [18]:
retrievers

{'finance': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021519ED9FD0>, search_kwargs={}),
 'human_resource': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021519ED8590>, search_kwargs={}),
 'marketing': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002152AD731E0>, search_kwargs={}),
 'engineering': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002151A2E0950>, search_kwargs={}),
 'employee': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002151A2E1190>, search_kwargs={})}

In [19]:
hr_retriever = retrievers['human_resource']

In [20]:
hr_retriever.invoke("Aadhya Patel")

[Document(id='5e7639e8-1a10-44c2-a371-477e233d0d79', metadata={'last_modified': '2025-06-05 13:54:51', 'row': 0, 'source': './resources/data\\hr\\hr_data.csv', 'role': 'hr'}, page_content='employee_id: FINEMP1000\nfull_name: Aadhya Patel\nrole: Sales Manager\ndepartment: Sales\nemail: aadhya.patel@fintechco.com\nlocation: Ahmedabad\ndate_of_birth: 1991-04-03\ndate_of_joining: 2018-11-20\nmanager_id: FINEMP1006\nsalary: 1332478.37\nleave_balance: 22\nleaves_taken: 11\nattendance_pct: 99.31\nperformance_rating: 3\nlast_review_date: 2024-05-21'),
 Document(id='6d6356e3-39a2-41bb-b75e-c94cdfed1970', metadata={'role': 'hr', 'last_modified': '2025-06-05 13:54:51', 'row': 0, 'source': './resources/data\\hr\\hr_data.csv'}, page_content='employee_id: FINEMP1000\nfull_name: Aadhya Patel\nrole: Sales Manager\ndepartment: Sales\nemail: aadhya.patel@fintechco.com\nlocation: Ahmedabad\ndate_of_birth: 1991-04-03\ndate_of_joining: 2018-11-20\nmanager_id: FINEMP1006\nsalary: 1332478.37\nleave_balance: 

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

In [22]:
prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based on the given context only.
    Do NOT answer anything outside of the context and reply with no results found if it is out of context
    provide the most accurate response based on the questions 
    <context>
    {context}
    </context>
    Question : {input}
    """
)

In [26]:
llm = ChatOpenAI()

In [27]:
stuff_document_chain=create_stuff_documents_chain(llm,prompt)

In [28]:
retrieval_chain = create_retrieval_chain(hr_retriever,stuff_document_chain)

In [29]:
result=retrieval_chain.invoke({"input": "What is the salary of Aadhya Patel?"})

In [30]:
result

{'input': 'What is the salary of Aadhya Patel?',
 'context': [Document(id='5e7639e8-1a10-44c2-a371-477e233d0d79', metadata={'source': './resources/data\\hr\\hr_data.csv', 'row': 0, 'role': 'hr', 'last_modified': '2025-06-05 13:54:51'}, page_content='employee_id: FINEMP1000\nfull_name: Aadhya Patel\nrole: Sales Manager\ndepartment: Sales\nemail: aadhya.patel@fintechco.com\nlocation: Ahmedabad\ndate_of_birth: 1991-04-03\ndate_of_joining: 2018-11-20\nmanager_id: FINEMP1006\nsalary: 1332478.37\nleave_balance: 22\nleaves_taken: 11\nattendance_pct: 99.31\nperformance_rating: 3\nlast_review_date: 2024-05-21'),
  Document(id='6d6356e3-39a2-41bb-b75e-c94cdfed1970', metadata={'role': 'hr', 'row': 0, 'last_modified': '2025-06-05 13:54:51', 'source': './resources/data\\hr\\hr_data.csv'}, page_content='employee_id: FINEMP1000\nfull_name: Aadhya Patel\nrole: Sales Manager\ndepartment: Sales\nemail: aadhya.patel@fintechco.com\nlocation: Ahmedabad\ndate_of_birth: 1991-04-03\ndate_of_joining: 2018-11-2

In [31]:
def create_chain_by_role(role):
    retriever = retrievers[role]
    retrieval_chain = create_retrieval_chain(retriever,stuff_document_chain)
    return retrieval_chain

In [35]:
hr_chain = create_chain_by_role("employee")

In [38]:
res = hr_chain.invoke({"input":"Salary of Aadhya Patel"})

In [39]:
res

{'input': 'Salary of Aadhya Patel',
 'context': [Document(id='c3e679bc-1a2b-4582-b6a0-9704d83c37d3', metadata={'role': 'general', 'last_modified': '2025-06-05 13:54:51', 'source': './resources/data\\general\\employee_handbook.md'}, page_content='| **House Rent Allowance (HRA)** | 40-50% of basic salary |\n| **Special Allowance** | Variable, as per grade |'),
  Document(id='84ad3ab0-5da7-4bea-b5fb-4c33700bd041', metadata={'source': './resources/data\\general\\employee_handbook.md', 'last_modified': '2025-06-05 13:54:51', 'role': 'general'}, page_content='| **House Rent Allowance (HRA)** | 40-50% of basic salary |\n| **Special Allowance** | Variable, as per grade |'),
  Document(id='754c884a-8993-4da7-8087-6528cbdea42e', metadata={'role': 'general', 'source': './resources/data\\general\\employee_handbook.md', 'last_modified': '2025-06-05 13:54:51'}, page_content='### Tuition & Certification Reimbursement\n- Up to ₹50,000/year for relevant courses, subject to manager and HR approval.'),
 

In [52]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

In [42]:
output_parser=StrOutputParser()

In [43]:
rewrite_query_system_template = (
    
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."

)

In [57]:
rewrite_query_prompt = ChatPromptTemplate.from_messages([
    ("system", rewrite_query_system_template),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [58]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")])

In [59]:
retrievers

{'finance': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021519ED9FD0>, search_kwargs={}),
 'human_resource': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021519ED8590>, search_kwargs={}),
 'marketing': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002152AD731E0>, search_kwargs={}),
 'engineering': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002151A2E0950>, search_kwargs={}),
 'employee': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002151A2E1190>, search_kwargs={})}

In [66]:
def rag_chain_by_role(model="gpt-4o-mini",role="employee"):
    llm = ChatOpenAI(model=model)
    history_aware_retriever = create_history_aware_retriever(llm=llm,prompt=rewrite_query_prompt,retriever=retrievers[role])
    qa_chain = create_stuff_documents_chain(llm,qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever,qa_chain)
    return rag_chain

In [76]:
emp_chain = rag_chain_by_role(role="employee")

In [77]:
chat_history = []

In [80]:
answer = emp_chain.invoke({
        "input": "Yearly sick leaves",
        "chat_history": chat_history
    })
answer

{'input': 'Yearly sick leaves',
 'chat_history': [],
 'context': [Document(id='4f4029f4-8d83-41dc-911d-f3cde2291efa', metadata={'role': 'general', 'last_modified': '2025-06-05 13:54:51', 'source': './resources/data\\general\\employee_handbook.md'}, page_content='| **Sick Leave** | 12 days/year (non-cumulative; medical certificate for >2 days) |\n| **Casual Leave** | 7 days/year (state-specific) |'),
  Document(id='a3fec24b-d9c6-457f-b1d2-c9d6b90f62f4', metadata={'source': './resources/data\\general\\employee_handbook.md', 'last_modified': '2025-06-05 13:54:51', 'role': 'general'}, page_content='| **Sick Leave** | 12 days/year (non-cumulative; medical certificate for >2 days) |\n| **Casual Leave** | 7 days/year (state-specific) |'),
  Document(id='31367e40-578e-4681-91a7-08234446c834', metadata={'source': './resources/data\\general\\employee_handbook.md', 'last_modified': '2025-06-05 13:54:51', 'role': 'general'}, page_content='- Sick leave for more than 2 days requires a medical certif

In [85]:
answer['context'][0]

Document(id='4f4029f4-8d83-41dc-911d-f3cde2291efa', metadata={'role': 'general', 'last_modified': '2025-06-05 13:54:51', 'source': './resources/data\\general\\employee_handbook.md'}, page_content='| **Sick Leave** | 12 days/year (non-cumulative; medical certificate for >2 days) |\n| **Casual Leave** | 7 days/year (state-specific) |')

In [88]:
answer['context'][0].metadata['source'].split("\\")[-1]

'employee_handbook.md'

In [92]:
source_docs = []
doc_len  = len(answer['context'])
for i in range(0,doc_len):
    source_docs.append(answer['context'][i].metadata['source'].split("\\")[-1])


In [93]:
source_docs

['employee_handbook.md',
 'employee_handbook.md',
 'employee_handbook.md',
 'employee_handbook.md']

In [94]:
import sys

def get_dict_size_mb(dictionary):
    size_bytes = sys.getsizeof(dictionary)
    size_mb = size_bytes / (1024 * 1024)
    return size_mb

In [95]:
size_in_mb = get_dict_size_mb(retrievers)
print(f"Size of the dictionary: {size_in_mb:.4f} MB")


Size of the dictionary: 0.0002 MB


Retrieval Chain Time analysis

In [1]:
from app.utils.database import init_db, get_chat_history, insert_application_logs

In [6]:
init_db()

In [8]:
import time

In [9]:
start = time.time()
chat_history = get_chat_history(session_id="721802cd-ea1e-43d3-b56d-da15d5e74dd3")
end = time.time()
print(end-start)

0.0009703636169433594


In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


def get_retriever_by_role(role):

    vectordb = Chroma(
        persist_directory="chroma_db",
        embedding_function=OpenAIEmbeddings(),
        collection_name=role
    )

    retriever = vectordb.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    return retriever


In [61]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from functools import cache
from dotenv import load_dotenv
load_dotenv()

rewrite_query_system_template = (
    
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."

)

rewrite_query_prompt = ChatPromptTemplate.from_messages([
    ("system", rewrite_query_system_template),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. The user may ask vague questions. Use the available documents and conversation history to answer precisely. If the question is too vague, politely ask for clarification."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")])

@cache
def rag_chain_by_role(model="gpt-4o-mini",role="employee"):
    llm = ChatOpenAI(model=model)
    history_aware_retriever = create_history_aware_retriever(llm=llm,prompt=rewrite_query_prompt,retriever=get_retriever_by_role(role))
    qa_chain = create_stuff_documents_chain(llm,qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever,qa_chain)
    return rag_chain

In [26]:
start = time.time()
rag = rag_chain_by_role(role="executive")
end = time.time()
print(end-start)

0.00010967254638671875


In [63]:
start = time.time()
answer = rag.invoke({
        "input": "Total Sick leaves in a year",
        "chat_history": chat_history
    })
end = time.time()
print(answer)
print(end-start)

{'input': 'Total Sick leaves in a year', 'chat_history': ['msg1', 'msg2', 'msg3', 'msg4', 'msg5', 'msg6', 'msg7', 'msg8', 'msg9', 'msg10'], 'context': [], 'answer': 'The total number of sick leaves in a year can vary significantly depending on company policy, local labor laws, and individual employment contracts. Typically, employers might offer anywhere from 5 to 15 sick days per year, with some providing additional leave for long-term illnesses or special circumstances. It’s best to check with the specific company’s HR policy for the exact number of sick leave days allowed.'}
4.402341604232788


In [50]:
messages = []
messages.extend([
    {"role":"human"},
    {"role": "ai"}
])

In [51]:
messages

[{'role': 'human'}, {'role': 'ai'}]

In [56]:
messages.extend([
    {"role":"human6"},
    {"role": "ai6"}
])

In [57]:
len(messages)

12

In [39]:
from math import floor

def trim_chat_history(chat_history):
    n = len(chat_history)
    if n == 0:
        return []

    # Calculate sizes
    k = max(1, floor(n * 0.2))  # At least 1 item per section

    # Indices for start, middle, and end
    first_k = chat_history[:k]
    
    # Middle 20% from the middle 60% section
    mid_start = n // 3
    mid_end = 2 * n // 3
    middle_section = chat_history[mid_start:mid_end]
    middle_k = middle_section[:k]

    # Last 20% of the whole list
    last_k = chat_history[-k:]

    # Combine results
    trimmed = first_k + middle_k + last_k
    return trimmed

In [59]:
messages

[{'role': 'human'},
 {'role': 'ai'},
 {'role': 'human2'},
 {'role': 'ai2'},
 {'role': 'human3'},
 {'role': 'ai3'},
 {'role': 'human4'},
 {'role': 'ai4'},
 {'role': 'human5'},
 {'role': 'ai5'},
 {'role': 'human6'},
 {'role': 'ai6'}]

In [60]:

trimmed = trim_chat_history(messages)
print(trimmed)

[{'role': 'human'}, {'role': 'ai'}, {'role': 'human3'}, {'role': 'ai3'}, {'role': 'human6'}, {'role': 'ai6'}]


In [ ]:
# Step 1 - load doc by role


print("===============================================================================================================")
print(f"Loading document for {role} ............................. ")
docs = load_documents_by_role(role)
print(f"Loading document for {role} complete")
print("===============================================================================================================")

# Step 2 - create vector store 
embedding=OpenAIEmbeddings()
persist_directory="chroma_db"


print(f"Creating Chroma vectorstore for collection: {role} ...... ")

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory,
    collection_name=role
)

# vectorstore.persist()
print(f"Creating Chroma vectorstore for collection: {role} success")
print("-----------------------------------------------------------------------------------------------------------------")
# return vectorstore

exe_vectorstore = Chroma(
    collection_name="executive",
    embedding=embedding,
    persist_directory=persist_directory
)

exe_vectorstore.add_documents(docs)
vectorstore.persist()

In [1]:
len("""Campaign Analysis:
-------------------------------------------
1. **Digital Campaigns**: FinSolve Technologies launched a series of digital campaigns, focusing on performance marketing, content marketing, and social media engagement. The **"InstantWire Global Expansion"** campaign, which highlighted FinSolve Technologies’s expansion into new markets, was the most successful in terms of conversions and brand awareness, leading to a 25% increase in traffic to the website and a 12% increase in sign-ups.
   - **Costs**: $5M spent on digital ads, influencer partnerships, and sponsored content.
   - **ROI**: 3.5x return on investment, with $17.5M in generated revenue directly attributed to these efforts.

2. **Event Marketing**: Several high-profile industry events and partnerships, including fintech expos and trade shows, helped solidify FinSolve Technologies’s presence in Europe and Asia. The **"FinSolve Technologies Fintech Innovation"** event in Berlin was the highlight, leading to over 300 new enterprise leads and strengthening vendor relationships.
   - **Costs**: $2M for event organization and travel.
   - **ROI**: Estimated 5x ROI from new partnerships and contracts.

3. **Email Marketing**: Email campaigns focused on retention and nurturing existing customers resulted in a 10% increase in customer retention. FinSolve Technologies sent over 2 million emails to existing customers, with a 25% open rate and a 15% click-through rate.
   - **Costs**: $200K for email design, segmentation, and automation tools.
   - **ROI**: 2x return on investment, with substantial increases in customer retention.""")

1620